Ejemplo tomado de https://github.com/StrikingLoo/Cats-and-dogs-classifier-tensorflow-CNN/blob/master/Convolutional_experiments.ipynb. El ejemplo ha sido simplificado y adaptado para el curso introduccion de aprendizaje de maquinas de la Maestria en Ingenieria de la Universidad del Magdalena

In [1]:
import tensorflow as tf
import seaborn as sns
import numpy as np

from PIL import Image
import glob
from collections import defaultdict
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
dir_path_cat = './PetImages\\Cat/*'
dir_path_dog = './PetImages\\Dog/*'
IMG_SIZE = (94, 125)

In [3]:
def pixels_from_path(file_path):
    im = Image.open(file_path)
    im = im.resize(IMG_SIZE)
    np_im = np.array(im)
    
    return np_im

In [4]:
cat_file_list = glob.glob(dir_path_cat) #Busca en el directorio dado todas las imagenes que alli se encuentran
dog_file_list = glob.glob(dir_path_dog)
print(len(cat_file_list))
print(len(dog_file_list))

12501
12501


In [5]:
valid_size = [IMG_SIZE[1], IMG_SIZE[0], 3] 

def create_set(file_list, nItems):
    new_set = []
    for im in file_list[:nItems]:
       new_item = pixels_from_path(im)
       if len(new_item.shape) != 3:
            print('Imagen con tamano diferente', im)
       else:
          new_set.append(new_item)
    
    return new_set   

In [6]:
SAMPLE_SIZE = 2048

print("loading training cat images...")
cat_train_set = create_set( cat_file_list, SAMPLE_SIZE)

print("loading training dog images...")
dog_train_set = create_set(dog_file_list, SAMPLE_SIZE)


loading training cat images...
Imagen con tamano diferente ./PetImages\Cat\10125.jpg
Imagen con tamano diferente ./PetImages\Cat\10501.jpg
Imagen con tamano diferente ./PetImages\Cat\10820.jpg
Imagen con tamano diferente ./PetImages\Cat\11095.jpg
Imagen con tamano diferente ./PetImages\Cat\11210.jpg
Imagen con tamano diferente ./PetImages\Cat\11565.jpg
loading training dog images...
Imagen con tamano diferente ./PetImages\Dog\10158.jpg
Imagen con tamano diferente ./PetImages\Dog\10401.jpg
Imagen con tamano diferente ./PetImages\Dog\10747.jpg
Imagen con tamano diferente ./PetImages\Dog\10797.jpg
Imagen con tamano diferente ./PetImages\Dog\11410.jpg
Imagen con tamano diferente ./PetImages\Dog\11675.jpg


In [7]:
print(len(cat_train_set))
print(len(dog_train_set))

2042
2042


In [8]:
TEST_SIZE = 512
cat_file_list_test =  cat_file_list[SAMPLE_SIZE+1:]
cat_test_set = create_set(cat_file_list_test, TEST_SIZE)

dog_file_list_test =  dog_file_list[SAMPLE_SIZE+1:]
dog_test_set = create_set(dog_file_list_test, TEST_SIZE)

Imagen con tamano diferente ./PetImages\Cat\11874.jpg
Imagen con tamano diferente ./PetImages\Cat\11935.jpg
Imagen con tamano diferente ./PetImages\Cat\12080.jpg
Imagen con tamano diferente ./PetImages\Dog\11849.jpg
Imagen con tamano diferente ./PetImages\Dog\11853.jpg


In [9]:
print(len(cat_test_set))
print(len(dog_test_set))

509
510


In [10]:
def validate_size(im_list):
   valid_size = (IMG_SIZE[1], IMG_SIZE[0], 3) 
   for im in im_list:
       im_shape = im.shape 
       #if not (np.array(im_shape) == np.array(valid_size)).all():
       if not (np.array_equal(np.array(im_shape), np.array(valid_size))):
          print('hay una imagen con un tamano:', im_shape)
          im_list.remove(im)
   return im_list

In [11]:
cat_train_set = validate_size(cat_train_set)
dog_train_set = validate_size(dog_train_set)

cat_test_set = validate_size(cat_test_set)
dog_test_set = validate_size(dog_test_set)

labels_cat_train = np.ones(len(cat_train_set))
labels_dog_train = np.zeros(len(dog_train_set))

labels_cat_test = np.ones(len(cat_test_set))
labels_dog_test = np.zeros(len(dog_test_set))

train_set =  cat_train_set  + dog_train_set
y_train = np.concatenate((labels_cat_train, labels_dog_train), axis=None)
 
test_set =  cat_test_set  + dog_test_set
y_test = np.concatenate((labels_cat_test, labels_dog_test), axis=None)

print(len(train_set))
print(len(test_set))
print(len(y_train))
print(len(y_test))

hay una imagen con un tamano: (125, 94, 4)
4083
1019
4083
1019


<ipython-input-10-eca6d32de37c>:8: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  im_list.remove(im)


In [12]:
X_train = np.array(train_set)
X_test = np.array(test_set)

print(X_train.shape)
print(X_test.shape)

(4083, 125, 94, 3)
(1019, 125, 94, 3)


# Un MLP normal

In [14]:
from tensorflow import keras
from tensorflow.keras import layers

img_size = IMG_SIZE

total_pixels = img_size[0] *img_size[1] * 3
fc_size = 512

#Definicion del modelo

inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image') 
x = layers.Flatten(name = 'flattened_img')(inputs) #turn image to vector.

x = layers.Dense(fc_size, activation='relu', name='first_layer')(x)
outputs = layers.Dense(1, activation='sigmoid', name='class')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [17]:
customAdam = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="mean_squared_error",
              # List of metrics to monitor
              metrics=["accuracy","mean_squared_error"])

In [18]:
print('# Fit model on training data')

history = model.fit(X_train, 
                    y_train,
                    batch_size=32, 
                    shuffle = True, #important since we loaded cats first, dogs second.
                    epochs=10,
                    validation_data=(X_test, y_test))

# Fit model on training data
Epoch 1/10
128/128 [==============================] - 7s 57ms/step - loss: 0.4999 - accuracy: 0.5001 - mean_squared_error: 0.4999 - val_loss: 0.5005 - val_accuracy: 0.4995 - val_mean_squared_error: 0.5005
Epoch 2/10
128/128 [==============================] - 7s 56ms/step - loss: 0.4999 - accuracy: 0.5001 - mean_squared_error: 0.4999 - val_loss: 0.5005 - val_accuracy: 0.4995 - val_mean_squared_error: 0.5005
Epoch 3/10
128/128 [==============================] - 7s 56ms/step - loss: 0.4999 - accuracy: 0.5001 - mean_squared_error: 0.4999 - val_loss: 0.5005 - val_accuracy: 0.4995 - val_mean_squared_error: 0.5005
Epoch 4/10
128/128 [==============================] - 7s 56ms/step - loss: 0.4999 - accuracy: 0.5001 - mean_squared_error: 0.4999 - val_loss: 0.5005 - val_accuracy: 0.4995 - val_mean_squared_error: 0.5005
Epoch 5/10
128/128 [==============================] - 7s 56ms/step - loss: 0.4999 - accuracy: 0.5001 - mean_squared_error: 0.4999 - val_loss: 0.5005 - 

# Single Convolutional Layer

In [20]:
fc_layer_size = 128
img_size = IMG_SIZE

conv_inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image')
conv_layer = layers.Conv2D(24, kernel_size=3, activation='relu')(conv_inputs)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)
conv_x = layers.Flatten(name = 'flattened_features')(conv_layer) #turn image to vector.

conv_x = layers.Dense(fc_layer_size, activation='relu', name='first_layer')(conv_x)
conv_x = layers.Dense(fc_layer_size, activation='relu', name='second_layer')(conv_x)
conv_outputs = layers.Dense(1, activation='sigmoid', name='class')(conv_x)

conv_model = keras.Model(inputs=conv_inputs, outputs=conv_outputs)

In [23]:
customAdam = keras.optimizers.Adam(lr=1e-6)
conv_model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="binary_crossentropy",
              # List of metrics to monitor
              metrics=["accuracy","mean_squared_error"])

In [35]:
print('# Fit model on training data')

history = conv_model.fit(X_train, 
                    y_train, #we pass it th labels
                    #If the model is taking forever to train, make this bigger
                    #If it is taking forever to load for the first epoch, make this smaller
                    batch_size=64, 
                    shuffle = True,
                    epochs=5,
                    # We pass it validation data to
                    # monitor loss and metrics
                    # at the end of each epoch
                    validation_data=(X_test, y_test))

# Fit model on training data
Epoch 1/5
64/64 [==============================] - 10s 150ms/step - loss: 1.3768 - accuracy: 0.5964 - mean_squared_error: 0.3226 - val_loss: 1.6542 - val_accuracy: 0.5613 - val_mean_squared_error: 0.3545
Epoch 2/5
64/64 [==============================] - 9s 148ms/step - loss: 1.2982 - accuracy: 0.6106 - mean_squared_error: 0.3103 - val_loss: 1.6013 - val_accuracy: 0.5643 - val_mean_squared_error: 0.3494
Epoch 3/5
64/64 [==============================] - 9s 147ms/step - loss: 1.2364 - accuracy: 0.6133 - mean_squared_error: 0.3033 - val_loss: 1.5630 - val_accuracy: 0.5790 - val_mean_squared_error: 0.3446
Epoch 4/5
64/64 [==============================] - 9s 148ms/step - loss: 1.1872 - accuracy: 0.6199 - mean_squared_error: 0.2967 - val_loss: 1.5507 - val_accuracy: 0.5741 - val_mean_squared_error: 0.3408
Epoch 5/5
64/64 [==============================] - 10s 151ms/step - loss: 1.1443 - accuracy: 0.6299 - mean_squared_error: 0.2894 - val_loss: 1.5087 - val_accu

# Bigger Convolutional Model

In [36]:
fc_layer_size = 256
img_size = IMG_SIZE

conv_inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image')

conv_layer = layers.Conv2D(48, kernel_size=3, activation='relu')(conv_inputs)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_layer = layers.Conv2D(48, kernel_size=3, activation='relu')(conv_layer)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_x = layers.Flatten(name = 'flattened_features')(conv_layer) #turn image to vector.

conv_x = layers.Dense(fc_layer_size, activation='relu', name='first_layer')(conv_x)
conv_x = layers.Dense(fc_layer_size, activation='relu', name='second_layer')(conv_x)
conv_outputs = layers.Dense(1, activation='sigmoid', name='class')(conv_x)

conv_model = keras.Model(inputs=conv_inputs, outputs=conv_outputs)

In [37]:
customAdam = keras.optimizers.Adam(lr=1e-6)
conv_model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="binary_crossentropy",
              # List of metrics to monitor
              metrics=["accuracy","mean_squared_error"])

In [38]:
print('# Fit model on training data')

history = conv_model.fit(X_train, 
                    y_train, 
                    batch_size=64, 
                    shuffle = True,
                    epochs=15,
                    # We pass it validation data to
                    # monitor loss and metrics
                    # at the end of each epoch
                    validation_data=(X_test, y_test))

# Fit model on training data
Epoch 1/15
64/64 [==============================] - 24s 372ms/step - loss: 2.1681 - accuracy: 0.4989 - mean_squared_error: 0.4251 - val_loss: 1.9135 - val_accuracy: 0.5074 - val_mean_squared_error: 0.4106
Epoch 2/15
64/64 [==============================] - 24s 378ms/step - loss: 1.6408 - accuracy: 0.5376 - mean_squared_error: 0.3772 - val_loss: 1.6676 - val_accuracy: 0.5280 - val_mean_squared_error: 0.3841
Epoch 3/15
64/64 [==============================] - 24s 372ms/step - loss: 1.3970 - accuracy: 0.5569 - mean_squared_error: 0.3478 - val_loss: 1.5349 - val_accuracy: 0.5417 - val_mean_squared_error: 0.3717
Epoch 4/15
64/64 [==============================] - 24s 374ms/step - loss: 1.2641 - accuracy: 0.5900 - mean_squared_error: 0.3259 - val_loss: 1.4391 - val_accuracy: 0.5456 - val_mean_squared_error: 0.3577
Epoch 5/15
64/64 [==============================] - 24s 372ms/step - loss: 1.1660 - accuracy: 0.5981 - mean_squared_error: 0.3107 - val_loss: 1.3956 - 

# Bigger Model2

In [32]:
fc_layer_size = 256
img_size = IMG_SIZE

conv_inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image')
conv_layer = layers.Conv2D(128, kernel_size=3, activation='relu')(conv_inputs)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_layer = layers.Conv2D(128, kernel_size=3, activation='relu')(conv_layer)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_x = layers.Flatten(name = 'flattened_features')(conv_layer) #turn image to vector.

conv_x = layers.Dense(fc_layer_size, activation='relu', name='first_layer')(conv_x)
conv_x = layers.Dense(fc_layer_size, activation='relu', name='second_layer')(conv_x)
conv_outputs = layers.Dense(1, activation='sigmoid', name='class')(conv_x)

huge_conv_model = keras.Model(inputs=conv_inputs, outputs=conv_outputs)

In [33]:
customAdam = keras.optimizers.Adam(lr=1e-6)
huge_conv_model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="binary_crossentropy",
              # List of metrics to monitor
              metrics=["binary_crossentropy","mean_squared_error"])

In [34]:
print('# Fit model on training data')

history = huge_conv_model.fit(X_train, 
                    y_train, #we pass it th labels
                    #If the model is taking forever to train, make this bigger
                    #If it is taking forever to load for the first epoch, make this smaller
                    batch_size=64, 
                    shuffle = True,
                    epochs=5,
                    # We pass it validation data to
                    # monitor loss and metrics
                    # at the end of each epoch
                    validation_data=(X_test, y_test))

# Fit model on training data
Epoch 1/5
64/64 [==============================] - 84s 1s/step - loss: 1.6947 - binary_crossentropy: 1.6947 - mean_squared_error: 0.3899 - val_loss: 1.5061 - val_binary_crossentropy: 1.5061 - val_mean_squared_error: 0.3801
Epoch 2/5
64/64 [==============================] - 85s 1s/step - loss: 1.1473 - binary_crossentropy: 1.1473 - mean_squared_error: 0.3209 - val_loss: 1.2750 - val_binary_crossentropy: 1.2750 - val_mean_squared_error: 0.3484
Epoch 3/5
64/64 [==============================] - 85s 1s/step - loss: 0.9690 - binary_crossentropy: 0.9690 - mean_squared_error: 0.2891 - val_loss: 1.1285 - val_binary_crossentropy: 1.1285 - val_mean_squared_error: 0.3276
Epoch 4/5
64/64 [==============================] - 84s 1s/step - loss: 0.8789 - binary_crossentropy: 0.8789 - mean_squared_error: 0.2712 - val_loss: 1.0607 - val_binary_crossentropy: 1.0607 - val_mean_squared_error: 0.3126
Epoch 5/5
64/64 [==============================] - 85s 1s/step - loss: 0.8113 -

# Modelo de Jason Brownlee

In [31]:
fc_layer_size = 256
img_size = IMG_SIZE

conv_inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image')

conv_layer = layers.Conv2D(64, kernel_size=3, activation='relu',
                           kernel_initializer='he_uniform')(conv_inputs)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_layer = layers.Dropout(0.2)(conv_layer)
conv_layer = layers.Conv2D(64, kernel_size=3, activation='relu',
                           kernel_initializer='he_uniform')(conv_layer)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_layer = layers.Dropout(0.2)(conv_layer)
conv_layer = layers.Conv2D(64, kernel_size=3, activation='relu',
                           kernel_initializer='he_uniform')(conv_layer)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_layer = layers.Dropout(0.2)(conv_layer)
conv_x = layers.Flatten(name = 'flattened_features')(conv_layer) #turn image to vector.

conv_x = layers.Dense(fc_layer_size, activation='relu', name='first_layer',
                      kernel_initializer='he_uniform')(conv_x)
conv_x = layers.Dropout(0.5)(conv_x)
conv_x = layers.Dense(fc_layer_size, activation='relu', name='second_layer')(conv_x)
conv_outputs = layers.Dense(1, activation='sigmoid', name='class')(conv_x)

Brownlee_model = keras.Model(inputs=conv_inputs, outputs=conv_outputs)


In [32]:
customAdam = keras.optimizers.Adam(lr=1e-6)
Brownlee_model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="binary_crossentropy",
              # List of metrics to monitor
              metrics=["binary_crossentropy","mean_squared_error"])

In [33]:
print('# Fit model on training data')

history = Brownlee_model.fit(X_train, 
                    y_train, #we pass it th labels
                    #If the model is taking forever to train, make this bigger
                    #If it is taking forever to load for the first epoch, make this smaller
                    batch_size=64, 
                    shuffle = True,
                    epochs=5,                             
                    # We pass it validation data to
                    # monitor loss and metrics
                    # at the end of each epoch
                    validation_data=(X_test, y_test))

# Fit model on training data
Epoch 1/5
64/64 [==============================] - 40s 617ms/step - loss: 148.5111 - binary_crossentropy: 148.5111 - mean_squared_error: 0.4880 - val_loss: 32.5123 - val_binary_crossentropy: 32.5123 - val_mean_squared_error: 0.4938
Epoch 2/5
64/64 [==============================] - 41s 643ms/step - loss: 128.4026 - binary_crossentropy: 128.4026 - mean_squared_error: 0.5074 - val_loss: 29.5024 - val_binary_crossentropy: 29.5024 - val_mean_squared_error: 0.4945
Epoch 3/5
64/64 [==============================] - 40s 632ms/step - loss: 120.4583 - binary_crossentropy: 120.4583 - mean_squared_error: 0.4959 - val_loss: 28.5973 - val_binary_crossentropy: 28.5973 - val_mean_squared_error: 0.4930
Epoch 4/5
64/64 [==============================] - 40s 629ms/step - loss: 112.6544 - binary_crossentropy: 112.6544 - mean_squared_error: 0.4951 - val_loss: 26.3280 - val_binary_crossentropy: 26.3280 - val_mean_squared_error: 0.4864
Epoch 5/5
64/64 [==========================

# Transfer learning

## VGG-16

In [ ]:
# define cnn model
import tensorflow as tf
from tensorflow import keras 

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

img_size = IMG_SIZE

model_VGG16 = VGG16(include_top=False, input_shape=(img_size[1], img_size[0],3))
for layer in model_VGG16.layers:
   layer.trainable = False

flat1  = Flatten()(model_VGG16.layers[-1].output)
class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
output = Dense(1, activation='sigmoid')(class1)

model_VGG16 = Model(inputs=model_VGG16.inputs, outputs=output)

customAdam = keras.optimizers.Adam(lr=1e-6)
model_VGG16.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="binary_crossentropy",
              # List of metrics to monitor
              metrics=["binary_crossentropy","mean_squared_error"])


history = model_VGG16.fit(X_train, 
                    y_train, #we pass it th labels
                    #If the model is taking forever to train, make this bigger
                    #If it is taking forever to load for the first epoch, make this smaller
                    batch_size = 64, 
                    shuffle = True,
                    epochs=10,                             
                    # We pass it validation data to
                    # monitor loss and metrics
                    # at the end of each epoch
                    validation_data=(X_test, y_test))


Epoch 1/10
64/64 [==============================] - 121s 2s/step - loss: 5.4601 - binary_crossentropy: 5.4601 - mean_squared_error: 0.3847 - val_loss: 5.3756 - val_binary_crossentropy: 5.3756 - val_mean_squared_error: 0.3758
Epoch 2/10
64/64 [==============================] - 121s 2s/step - loss: 4.8114 - binary_crossentropy: 4.8114 - mean_squared_error: 0.3669 - val_loss: 4.7586 - val_binary_crossentropy: 4.7586 - val_mean_squared_error: 0.3695
Epoch 3/10
64/64 [==============================] - 122s 2s/step - loss: 4.3781 - binary_crossentropy: 4.3781 - mean_squared_error: 0.3536 - val_loss: 4.3236 - val_binary_crossentropy: 4.3236 - val_mean_squared_error: 0.3608
Epoch 4/10
64/64 [==============================] - 120s 2s/step - loss: 4.0675 - binary_crossentropy: 4.0675 - mean_squared_error: 0.3430 - val_loss: 4.0012 - val_binary_crossentropy: 4.0012 - val_mean_squared_error: 0.3480
Epoch 5/10
64/64 [==============================] - 122s 2s/step - loss: 3.8242 - binary_crossentrop